In [1]:
import pandas as pd
import numpy as np
import geopandas as gp

# ---------------------------------
# Biz_Cts_2022_Base_Yr

In [2]:
biz_v1 = pd.read_csv(r"J:\DataScience\DSEconProdDessem\EC2\Biz_Cts_2022_Base_Yr\Outputs\Biz_Cts_by_EmpCtr2_for_baseyear_2022.csv")
biz_v1.head()

,Unnamed: 0,EC_ID,EC_Name,Tier,Gvt,Mil,Ag_Min,Bus_Svcs,Fin_Res_Mgt,Educ,Hlth_Care,Retail,Trn_Wrh_Cons,Utl_Mnf_Whl,Entertn,Accomid,Food,Other,Total
0,0,0,NaN,0,345,2,363,5898,2792,952,13423,1564,3913,1376,477,108,1250,2674,35137
1,1,1,San Diego Airport,3,6,0,3,196,103,26,53,91,86,49,34,26,110,74,857
2,2,2,Alpine,4,6,0,0,21,20,4,25,22,24,2,3,1,28,11,167
3,3,3,Barrio Logan,4,3,0,0,24,10,7,65,37,23,52,1,2,41,23,288
4,4,4,Carlsbad Palomar Airport,2,13,0,4,515,236,29,68,87,82,347,24,9,46,45,1505


In [3]:
biz_v2 = pd.read_csv(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Biz_Cts_2022_Base_Yr\Outputs\biz_cts_2022.csv')
biz_v2.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Regional_Count,Regional_Percent,Geometry
0,1,San Diego Airport,3,0,Gvt,Biz by Industry,6,0.70,1184,1.16,MULTIPOLYGON (((6266765.922053143 1846488.1797...
1,2,Alpine,4,0,Gvt,Biz by Industry,6,3.59,1184,1.16,MULTIPOLYGON (((6409937.99988164 1889070.00005...
2,3,Barrio Logan,4,0,Gvt,Biz by Industry,3,1.04,1184,1.16,MULTIPOLYGON (((6286603.12483272 1836426.15324...
3,4,Carlsbad Palomar Airport,2,0,Gvt,Biz by Industry,13,0.86,1184,1.16,MULTIPOLYGON (((6252306.248872802 1998205.2030...
4,5,Carlsbad State Beach,3,0,Gvt,Biz by Industry,3,0.58,1184,1.16,MULTIPOLYGON (((6231083.038038477 1994552.5840...


In [4]:
biz_geojson = gp.read_file(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Biz_Cts_2022_Base_Yr\Outputs\biz_cts_2022.geojson')
biz_geojson.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Regional_Count,Regional_Percent,geometry
0,1,San Diego Airport,3,0,Gvt,Biz by Industry,6,0.70,1184,1.16,"MULTIPOLYGON (((6266765.922 1846488.180, 62667..."
1,2,Alpine,4,0,Gvt,Biz by Industry,6,3.59,1184,1.16,"MULTIPOLYGON (((6409938.000 1889070.000, 64098..."
2,3,Barrio Logan,4,0,Gvt,Biz by Industry,3,1.04,1184,1.16,"MULTIPOLYGON (((6286603.125 1836426.153, 62866..."
3,4,Carlsbad Palomar Airport,2,0,Gvt,Biz by Industry,13,0.86,1184,1.16,"MULTIPOLYGON (((6252306.249 1998205.203, 62523..."
4,5,Carlsbad State Beach,3,0,Gvt,Biz by Industry,3,0.58,1184,1.16,"MULTIPOLYGON (((6231083.038 1994552.584, 62311..."


In [5]:
# check all ECs in v2
for ec in biz_v1['EC_Name'].unique():
    if ec not in biz_v2['EC_Name'].unique():
        print(ec)

# check all variables in v2
for var in biz_v1.columns:
    if var not in biz_v2['Variable'].unique():
        print(var)

nan
Unnamed: 0
EC_ID
EC_Name
Tier
Total


In [6]:
# check totals by EC
biz_v2.groupby(['EC_ID', 'EC_Name', 'Tier', 'Parent']).sum()['Count'].to_list() == biz_v1['Total'][1:].to_list()

True

In [7]:
# check totals by variable
biz_v2.groupby('Variable')['Count'].sum().sort_index()==biz_v1.iloc[1:][biz_v1.columns[4:-1]].sum().sort_index()

Variable
Accomid         True
Ag_Min          True
Bus_Svcs        True
Educ            True
Entertn         True
Fin_Res_Mgt     True
Food            True
Gvt             True
Hlth_Care       True
Mil             True
Other           True
Retail          True
Trn_Wrh_Cons    True
Utl_Mnf_Whl     True
dtype: bool

In [8]:
# compare with geojson
(biz_v2[biz_v2.columns[:-1]] == biz_geojson[biz_geojson.columns[:-1]]).sum()

EC_ID               1428
EC_Name             1428
Tier                1428
Parent              1428
Variable            1428
Category            1428
Count               1428
Percent             1428
Regional_Count      1428
Regional_Percent    1428
dtype: int64

In [9]:
# check percentages
totals_biz = biz_v2.groupby('EC_Name')['Count'].sum().reset_index().merge(biz_v2, on='EC_Name', how='outer', suffixes=('_totals', '_counts'))
(totals_biz['Percent'] == round(totals_biz['Count_counts']/totals_biz['Count_totals']*100, 2)).sum()

1428

# ---------------------------------
# Dem-Data_by_EC_yr-2022

## 1. Number of households

In [10]:
households_v1 = pd.read_csv(r"J:\DataScience\DSEconProdDessem\EC2\Dem_Data by_EC yr_2022\Outputs\Number_of_HouseHolds.csv")
households_v1.head()

,EC_ID,EC_Name,Tier,Parent,HH,HH_SF,HH_MF,HH_MH,%HH_SF,%HH_MF,%HH_MH
0,1,San Diego Airport,3,0,2585,933,1652,0,36.1,63.9,0.0
1,2,Alpine,4,0,1608,840,622,146,52.2,38.7,9.1
2,3,Barrio Logan,4,0,1563,761,802,0,48.7,51.3,0.0
3,4,Carlsbad Palomar Airport,2,0,336,92,244,0,27.4,72.6,0.0
4,5,Carlsbad State Beach,3,0,345,0,252,93,0.0,73.0,27.0


In [11]:
households_v2 = pd.read_csv(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Dem-Data_by_EC_yr-2022\Outputs\.csv files\number_of_housholds.csv')
households_v2.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Reginal_Counts,Reginal_Percents,Geometry
0,1,San Diego Airport,3,0,HH_SF,Households by HU Type,933,36.1,693899.0,59.8,MULTIPOLYGON (((6266765.922053143 1846488.1797...
1,2,Alpine,4,0,HH_SF,Households by HU Type,840,52.2,693899.0,59.8,MULTIPOLYGON (((6409937.99988164 1889070.00005...
2,3,Barrio Logan,4,0,HH_SF,Households by HU Type,761,48.7,693899.0,59.8,MULTIPOLYGON (((6286603.12483272 1836426.15324...
3,4,Carlsbad Palomar Airport,2,0,HH_SF,Households by HU Type,92,27.4,693899.0,59.8,MULTIPOLYGON (((6252306.248872802 1998205.2030...
4,5,Carlsbad State Beach,3,0,HH_SF,Households by HU Type,0,0.0,693899.0,59.8,MULTIPOLYGON (((6231083.038038477 1994552.5840...


In [12]:
households_geojson = gp.read_file(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Dem-Data_by_EC_yr-2022\Outputs\.geojson files\number_of_households.geojson')
households_geojson.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Reginal_Counts,Reginal_Percents,geometry
0,1,San Diego Airport,3,0,HH_SF,Households by HU Type,933,36.1,693899.0,59.8,"MULTIPOLYGON (((6266765.92205 1846488.17971, 6..."
1,2,Alpine,4,0,HH_SF,Households by HU Type,840,52.2,693899.0,59.8,"MULTIPOLYGON (((6409937.99988 1889070.00006, 6..."
2,3,Barrio Logan,4,0,HH_SF,Households by HU Type,761,48.7,693899.0,59.8,"MULTIPOLYGON (((6286603.12483 1836426.15325, 6..."
3,4,Carlsbad Palomar Airport,2,0,HH_SF,Households by HU Type,92,27.4,693899.0,59.8,"MULTIPOLYGON (((6252306.24887 1998205.20305, 6..."
4,5,Carlsbad State Beach,3,0,HH_SF,Households by HU Type,0,0.0,693899.0,59.8,"MULTIPOLYGON (((6231083.03804 1994552.58404, 6..."


In [13]:
# check all ECs in v2
for ec in households_v1['EC_Name'].unique():
    if ec not in households_v2['EC_Name'].unique():
        print(ec)

# check all variables in v2
for var in households_v1.columns:
    if var not in households_v2['Variable'].unique():
        print(var)

Carlsbad Palomar Airport Sub-Center: Airport
Carlsbad Palomar Airport Sub-Center: El Camino Real
Carlsbad Palomar Airport Sub-Center: Faraday
Carlsbad Palomar Airport Sub-Center: Melrose
Carlsbad Palomar Airport Sub-Center: Palomar Oaks
Kearny Mesa East Sub-Center: Aero Dr
Kearny Mesa East Sub-Center: Balboa Ave
Kearny Mesa East Sub-Center: Cabrillo Hts
Kearny Mesa East Sub-Center: I-15 Border
Kearny Mesa East Sub-Center: Montgomery-Gibbs Exec Airport
Kearny Mesa East Sub-Center: North Point
Kearny Mesa East Sub-Center: Spectrum Ctr
Sorrento Valley West Sub-Center: Scripps Research
EC_ID
EC_Name
Tier
Parent
HH
%HH_SF
%HH_MF
%HH_MH


In [14]:
# check totals by EC
households_v1[households_v1['EC_Name'].isin(households_v2['EC_Name'].unique())].sort_values('EC_Name')['HH'].to_list() == households_v2.groupby(['EC_Name', 'Tier', 'Parent']).sum()['Count'].to_list()

True

In [15]:
# check totals by variable
households_v2.groupby('Variable')['Count'].sum().sort_index() == households_v1[households_v1['EC_Name'].isin(households_v2['EC_Name'].unique())][households_v1.columns[5:8]].sum().sort_index()

Variable
HH_MF    True
HH_MH    True
HH_SF    True
dtype: bool

In [16]:
# compare with geojson
(households_v2[households_v2.columns[:-1]] == households_geojson[households_geojson.columns[:-1]]).sum()

EC_ID               396
EC_Name             396
Tier                396
Parent              396
Variable            396
Category            396
Count               396
Percent             396
Reginal_Counts      396
Reginal_Percents    396
dtype: int64

In [17]:
# check percentages
totals_households = households_v2.groupby('EC_Name')['Count'].sum().reset_index().merge(households_v2, on='EC_Name', how='outer', suffixes=('_totals', '_counts'))
totals_households[~(totals_households['Percent'] == round(totals_households['Count_counts']/totals_households['Count_totals']*100, 1).replace(np.nan, 0))]

,EC_Name,Count_totals,EC_ID,Tier,Parent,Variable,Category,Count_counts,Percent,Reginal_Counts,Reginal_Percents,Geometry


## 2. Number of household units

In [18]:
housing_units_v1 = pd.read_csv(r"J:\DataScience\DSEconProdDessem\EC2\Dem_Data by_EC yr_2022\Outputs\Number_of_housing-units.csv")
housing_units_v1.head()

,EC_ID,EC_Name,Tier,Parent,HU_SF,HU_SFMU,HU_MF,HU_MH,HU_Total,%HU_SF,%HU_SFMU,%HU_MF,%HU_MH
0,1,San Diego Airport,3,0,458,556,1825,0,2839,16.1,19.6,64.3,0.0
1,2,Alpine,4,0,808,62,654,154,1678,48.2,3.7,39.0,9.2
2,3,Barrio Logan,4,0,286,515,917,0,1718,16.6,30.0,53.4,0.0
3,4,Carlsbad Palomar Airport,2,0,46,51,245,0,342,13.5,14.9,71.6,0.0
4,5,Carlsbad State Beach,3,0,0,0,273,108,381,0.0,0.0,71.7,28.3


In [19]:
housing_units_v2 = pd.read_csv(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Dem-Data_by_EC_yr-2022\Outputs\.csv files\number_of_housing_units.csv')
housing_units_v2.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Reginal_Counts,Reginal_Percents,Geometry
0,1,San Diego Airport,3,0,HU_SF,Housing Units by Type,458,16.1,571452.0,46.3,MULTIPOLYGON (((6266765.922053143 1846488.1797...
1,2,Alpine,4,0,HU_SF,Housing Units by Type,808,48.2,571452.0,46.3,MULTIPOLYGON (((6409937.99988164 1889070.00005...
2,3,Barrio Logan,4,0,HU_SF,Housing Units by Type,286,16.6,571452.0,46.3,MULTIPOLYGON (((6286603.12483272 1836426.15324...
3,4,Carlsbad Palomar Airport,2,0,HU_SF,Housing Units by Type,46,13.5,571452.0,46.3,MULTIPOLYGON (((6252306.248872802 1998205.2030...
4,5,Carlsbad State Beach,3,0,HU_SF,Housing Units by Type,0,0.0,571452.0,46.3,MULTIPOLYGON (((6231083.038038477 1994552.5840...


In [20]:
housing_units_geojson = gp.read_file(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Dem-Data_by_EC_yr-2022\Outputs\.geojson files\number_of_housing_units.geojson')
housing_units_geojson.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Reginal_Counts,Reginal_Percents,geometry
0,1,San Diego Airport,3,0,HU_SF,Housing Units by Type,458,16.1,571452.0,46.3,"MULTIPOLYGON (((6266765.92205 1846488.17971, 6..."
1,2,Alpine,4,0,HU_SF,Housing Units by Type,808,48.2,571452.0,46.3,"MULTIPOLYGON (((6409937.99988 1889070.00006, 6..."
2,3,Barrio Logan,4,0,HU_SF,Housing Units by Type,286,16.6,571452.0,46.3,"MULTIPOLYGON (((6286603.12483 1836426.15325, 6..."
3,4,Carlsbad Palomar Airport,2,0,HU_SF,Housing Units by Type,46,13.5,571452.0,46.3,"MULTIPOLYGON (((6252306.24887 1998205.20305, 6..."
4,5,Carlsbad State Beach,3,0,HU_SF,Housing Units by Type,0,0.0,571452.0,46.3,"MULTIPOLYGON (((6231083.03804 1994552.58404, 6..."


In [21]:
# check all ECs in v2
for ec in housing_units_v1['EC_Name'].unique():
    if ec not in housing_units_v2['EC_Name'].unique():
        print(ec)

# check all variables in v2
for var in housing_units_v1.columns:
    if var not in housing_units_v2['Variable'].unique():
        print(var)

Carlsbad Palomar Airport Sub-Center: Airport
Carlsbad Palomar Airport Sub-Center: El Camino Real
Carlsbad Palomar Airport Sub-Center: Faraday
Carlsbad Palomar Airport Sub-Center: Melrose
Carlsbad Palomar Airport Sub-Center: Palomar Oaks
Kearny Mesa East Sub-Center: Aero Dr
Kearny Mesa East Sub-Center: Balboa Ave
Kearny Mesa East Sub-Center: Cabrillo Hts
Kearny Mesa East Sub-Center: I-15 Border
Kearny Mesa East Sub-Center: Montgomery-Gibbs Exec Airport
Kearny Mesa East Sub-Center: North Point
Kearny Mesa East Sub-Center: Spectrum Ctr
Sorrento Valley West Sub-Center: Scripps Research
EC_ID
EC_Name
Tier
Parent
HU_Total
%HU_SF
%HU_SFMU
%HU_MF
%HU_MH


In [22]:
# check totals by EC
housing_units_v1[housing_units_v1['EC_Name'].isin(housing_units_v2['EC_Name'].unique())].sort_values('EC_Name')['HU_Total'].to_list() == housing_units_v2.groupby(['EC_Name', 'Tier', 'Parent']).sum()['Count'].to_list()

True

In [23]:
# check totals by variable
housing_units_v2.groupby('Variable')['Count'].sum().sort_index() == housing_units_v1[housing_units_v1['EC_Name'].isin(housing_units_v2['EC_Name'].unique())][housing_units_v1.columns[4:8]].sum().sort_index()

Variable
HU_MF      True
HU_MH      True
HU_SF      True
HU_SFMU    True
dtype: bool

In [24]:
# compare with geojson
print(housing_units_v2.shape)
(housing_units_v2[housing_units_v2.columns[:-1]] == housing_units_geojson[housing_units_geojson.columns[:-1]]).sum()

(528, 11)


EC_ID               528
EC_Name             528
Tier                528
Parent              528
Variable            528
Category            528
Count               528
Percent             528
Reginal_Counts      528
Reginal_Percents    528
dtype: int64

In [25]:
# check percentages
totals_hu = housing_units_v2.groupby('EC_Name')['Count'].sum().reset_index().merge(housing_units_v2, on='EC_Name', how='outer', suffixes=('_totals', '_counts'))
totals_hu[~(totals_hu['Percent'] == round(totals_hu['Count_counts']/totals_hu['Count_totals']*100, 1).replace(np.nan, 0))]

,EC_Name,Count_totals,EC_ID,Tier,Parent,Variable,Category,Count_counts,Percent,Reginal_Counts,Reginal_Percents,Geometry


## 3. Population by Age

In [26]:
pop_age_v1 = pd.read_csv(r"J:\DataScience\DSEconProdDessem\EC2\Dem_Data by_EC yr_2022\Outputs\pop_by_age.csv")
pop_age_v1.head()

,EC_ID,EC_Name,Tier,Parent,Pop_0-4_Yrs,Pop_5-9_Yrs,Pop_10-14_Yrs,Pop_15-17_Yrs,Pop_18-19_Yrs,Pop_20-24_Yrs,...,%Pop_45-49_Yrs,%Pop_50-54_Yrs,%Pop_55-59_Yrs,%Pop_60-61_Yrs,%Pop_62-64_Yrs,%Pop_65-69_Yrs,%Pop_70-74_Yrs,%Pop_75-79_Yrs,%Pop_80-84_Yrs,%Pop_85+_Yrs
0,1,San Diego Airport,3,0,318,367,263,156,94,374,...,6.1,4.8,4.5,1.5,2.3,6.1,5.8,4.3,2.4,2.1
1,2,Alpine,4,0,247,251,331,200,107,218,...,7.4,6.8,5.1,1.8,3.4,7.3,5.6,5.5,2.8,1.9
2,3,Barrio Logan,4,0,409,428,262,167,160,328,...,7.5,6.0,4.2,1.5,2.7,3.9,3.1,1.5,1.2,2.3
3,4,Carlsbad Palomar Airport,2,0,27,71,77,67,35,59,...,13.1,7.6,6.1,3.2,2.1,3.9,2.3,3.1,2.3,2.7
4,5,Carlsbad State Beach,3,0,46,65,51,25,14,26,...,6.7,4.0,9.1,5.8,3.5,4.3,4.8,6.1,1.9,1.8


In [27]:
pop_age_v2 = pd.read_csv(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Dem-Data_by_EC_yr-2022\Outputs\.csv files\pop_by_age.csv')
pop_age_v2.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Reginal_Counts,Reginal_Percents,Geometry
0,1,San Diego Airport,3,0,0-4_Yrs,Pop by Age,318,5.5,187752.0,5.7,MULTIPOLYGON (((6266765.922053143 1846488.1797...
1,2,Alpine,4,0,0-4_Yrs,Pop by Age,247,5.7,187752.0,5.7,MULTIPOLYGON (((6409937.99988164 1889070.00005...
2,3,Barrio Logan,4,0,0-4_Yrs,Pop by Age,409,8.1,187752.0,5.7,MULTIPOLYGON (((6286603.12483272 1836426.15324...
3,4,Carlsbad Palomar Airport,2,0,0-4_Yrs,Pop by Age,27,2.7,187752.0,5.7,MULTIPOLYGON (((6252306.248872802 1998205.2030...
4,5,Carlsbad State Beach,3,0,0-4_Yrs,Pop by Age,46,5.5,187752.0,5.7,MULTIPOLYGON (((6231083.038038477 1994552.5840...


In [28]:
pop_age_geojson = gp.read_file(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Dem-Data_by_EC_yr-2022\Outputs\.geojson files\pop_by_age.geojson')
pop_age_geojson.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Reginal_Counts,Reginal_Percents,geometry
0,1,San Diego Airport,3,0,0-4_Yrs,Pop by Age,318,5.5,187752.0,5.7,"MULTIPOLYGON (((6266765.92205 1846488.17971, 6..."
1,2,Alpine,4,0,0-4_Yrs,Pop by Age,247,5.7,187752.0,5.7,"MULTIPOLYGON (((6409937.99988 1889070.00006, 6..."
2,3,Barrio Logan,4,0,0-4_Yrs,Pop by Age,409,8.1,187752.0,5.7,"MULTIPOLYGON (((6286603.12483 1836426.15325, 6..."
3,4,Carlsbad Palomar Airport,2,0,0-4_Yrs,Pop by Age,27,2.7,187752.0,5.7,"MULTIPOLYGON (((6252306.24887 1998205.20305, 6..."
4,5,Carlsbad State Beach,3,0,0-4_Yrs,Pop by Age,46,5.5,187752.0,5.7,"MULTIPOLYGON (((6231083.03804 1994552.58404, 6..."


In [29]:
# check all ECs in v2
for ec in pop_age_v1['EC_Name'].unique():
    if ec not in pop_age_v2['EC_Name'].unique():
        print(ec)

# check all variables in v2
for var in pop_age_v1.columns:
    if var not in pop_age_v2['Variable'].unique():
        if '_'.join(var.split('_')[1:]) not in pop_age_v2['Variable'].unique():
            print(var)

Carlsbad Palomar Airport Sub-Center: Airport
Carlsbad Palomar Airport Sub-Center: El Camino Real
Carlsbad Palomar Airport Sub-Center: Faraday
Carlsbad Palomar Airport Sub-Center: Melrose
Carlsbad Palomar Airport Sub-Center: Palomar Oaks
Kearny Mesa East Sub-Center: Aero Dr
Kearny Mesa East Sub-Center: Balboa Ave
Kearny Mesa East Sub-Center: Cabrillo Hts
Kearny Mesa East Sub-Center: I-15 Border
Kearny Mesa East Sub-Center: Montgomery-Gibbs Exec Airport
Kearny Mesa East Sub-Center: North Point
Kearny Mesa East Sub-Center: Spectrum Ctr
Sorrento Valley West Sub-Center: Scripps Research
EC_ID
EC_Name
Tier
Parent


In [30]:
# check totals by EC
pop_age_v1[pop_age_v1['EC_Name'].isin(pop_age_v2['EC_Name'].unique())].sort_values('EC_Name')[pop_age_v1.columns[4:24]].sum(axis=1).to_list() == pop_age_v2.groupby(['EC_Name', 'Tier', 'Parent']).sum()['Count'].to_list()

True

In [31]:
# check totals by variable
pop_age_v2.groupby('Variable')['Count'].sum().sort_index().to_list() == pop_age_v1[pop_age_v1['EC_Name'].isin(pop_age_v2['EC_Name'].unique())][pop_age_v1.columns[4:24]].sum().sort_index().to_list()

True

In [32]:
# check percentages
totals_pop_age = pop_age_v2.groupby('EC_Name')['Count'].sum().reset_index().merge(pop_age_v2, on='EC_Name', how='outer', suffixes=('_totals', '_counts'))
totals_pop_age[~(totals_pop_age['Percent'] == round(totals_pop_age['Count_counts']/totals_pop_age['Count_totals']*100, 1).replace(np.nan, 0))]

,EC_Name,Count_totals,EC_ID,Tier,Parent,Variable,Category,Count_counts,Percent,Reginal_Counts,Reginal_Percents,Geometry


In [33]:
# compare with geojson
print(pop_age_v2.shape)
(pop_age_v2[pop_age_v2.columns[:-1]] == pop_age_geojson[pop_age_geojson.columns[:-1]]).sum()

(2640, 11)


EC_ID               2640
EC_Name             2640
Tier                2640
Parent              2640
Variable            2640
Category            2640
Count               2640
Percent             2640
Reginal_Counts      2640
Reginal_Percents    2640
dtype: int64

## 4. Population by Race and Ethnicity

In [34]:
pop_race_v1 = pd.read_csv(r"J:\DataScience\DSEconProdDessem\EC2\Dem_Data by_EC yr_2022\Outputs\pop_by_ethnicity-race.csv")
pop_race_v1.head()

,EC_ID,EC_Name,Tier,Parent,Pop_Hisp,Pop_Wht,Pop_Blk,Pop_Am-Ind,Pop_Asn,Pop_Pac-Isl,Pop_Oth,Pop_2+,%Pop_Hisp,%Pop_Wht,%Pop_Blk,%Pop_Am-Ind,%Pop_Asn,%Pop_Pac-Isl,%Pop_Oth,%Pop_2+
0,1,San Diego Airport,3,0,1325,3362,244,21,344,60,92,317,23.0,58.3,4.2,0.4,6.0,1.0,1.6,5.5
1,2,Alpine,4,0,672,3087,108,2,221,4,0,270,15.4,70.7,2.5,0.0,5.1,0.1,0.0,6.2
2,3,Barrio Logan,4,0,3727,860,260,11,168,18,1,21,73.6,17.0,5.1,0.2,3.3,0.4,0.0,0.4
3,4,Carlsbad Palomar Airport,2,0,223,509,17,0,213,0,0,49,22.1,50.3,1.7,0.0,21.1,0.0,0.0,4.8
4,5,Carlsbad State Beach,3,0,113,531,3,0,125,3,0,56,13.6,63.9,0.4,0.0,15.0,0.4,0.0,6.7


In [35]:
pop_race_v2 = pd.read_csv(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Dem-Data_by_EC_yr-2022\Outputs\.csv files\pop_by_race_ethnicity.csv')
pop_race_v2.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Reginal_Counts,Reginal_Percents,Geometry
0,1,San Diego Airport,3,0,Hisp,Pop by Race-Eth,1325,23.0,1122382.0,34.1,MULTIPOLYGON (((6266765.922053143 1846488.1797...
1,2,Alpine,4,0,Hisp,Pop by Race-Eth,672,15.4,1122382.0,34.1,MULTIPOLYGON (((6409937.99988164 1889070.00005...
2,3,Barrio Logan,4,0,Hisp,Pop by Race-Eth,3727,73.6,1122382.0,34.1,MULTIPOLYGON (((6286603.12483272 1836426.15324...
3,4,Carlsbad Palomar Airport,2,0,Hisp,Pop by Race-Eth,223,22.1,1122382.0,34.1,MULTIPOLYGON (((6252306.248872802 1998205.2030...
4,5,Carlsbad State Beach,3,0,Hisp,Pop by Race-Eth,113,13.6,1122382.0,34.1,MULTIPOLYGON (((6231083.038038477 1994552.5840...


In [36]:
pop_race_geojson = gp.read_file(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Dem-Data_by_EC_yr-2022\Outputs\.geojson files\pob_by_race_ethnicity.geojson')
pop_race_geojson.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Reginal_Counts,Reginal_Percents,geometry
0,1,San Diego Airport,3,0,Hisp,Pop by Race-Eth,1325,23.0,1122382.0,34.1,"MULTIPOLYGON (((6266765.92205 1846488.17971, 6..."
1,2,Alpine,4,0,Hisp,Pop by Race-Eth,672,15.4,1122382.0,34.1,"MULTIPOLYGON (((6409937.99988 1889070.00006, 6..."
2,3,Barrio Logan,4,0,Hisp,Pop by Race-Eth,3727,73.6,1122382.0,34.1,"MULTIPOLYGON (((6286603.12483 1836426.15325, 6..."
3,4,Carlsbad Palomar Airport,2,0,Hisp,Pop by Race-Eth,223,22.1,1122382.0,34.1,"MULTIPOLYGON (((6252306.24887 1998205.20305, 6..."
4,5,Carlsbad State Beach,3,0,Hisp,Pop by Race-Eth,113,13.6,1122382.0,34.1,"MULTIPOLYGON (((6231083.03804 1994552.58404, 6..."


In [37]:
# check all ECs in v2
for ec in pop_race_v1['EC_Name'].unique():
    if ec not in pop_race_v2['EC_Name'].unique():
        print(ec)

# check all variables in v2
for var in pop_race_v1.columns:
    if var not in pop_race_v2['Variable'].unique():
        if '_'.join(var.split('_')[1:]) not in pop_race_v2['Variable'].unique():
            print(var)

Carlsbad Palomar Airport Sub-Center: Airport
Carlsbad Palomar Airport Sub-Center: El Camino Real
Carlsbad Palomar Airport Sub-Center: Faraday
Carlsbad Palomar Airport Sub-Center: Melrose
Carlsbad Palomar Airport Sub-Center: Palomar Oaks
Kearny Mesa East Sub-Center: Aero Dr
Kearny Mesa East Sub-Center: Balboa Ave
Kearny Mesa East Sub-Center: Cabrillo Hts
Kearny Mesa East Sub-Center: I-15 Border
Kearny Mesa East Sub-Center: Montgomery-Gibbs Exec Airport
Kearny Mesa East Sub-Center: North Point
Kearny Mesa East Sub-Center: Spectrum Ctr
Sorrento Valley West Sub-Center: Scripps Research
EC_ID
EC_Name
Tier
Parent


In [38]:
# check totals by EC
pop_race_v1[pop_race_v1['EC_Name'].isin(pop_race_v2['EC_Name'].unique())].sort_values('EC_Name')[pop_race_v1.columns[4:12]].sum(axis=1).to_list() == pop_race_v2.groupby(['EC_Name', 'Tier', 'Parent']).sum()['Count'].to_list()

True

In [39]:
# check totals by variable
pop_race_v2.groupby('Variable')['Count'].sum().sort_index().to_list() == pop_race_v1[pop_race_v1['EC_Name'].isin(pop_race_v2['EC_Name'].unique())][pop_race_v1.columns[4:12]].sum().sort_index().to_list()

True

In [40]:
# check percentages
totals_pop_race = pop_race_v2.groupby('EC_Name')['Count'].sum().reset_index().merge(pop_race_v2, on='EC_Name', how='outer', suffixes=('_totals', '_counts'))
totals_pop_race[~(totals_pop_race['Percent'] == round(totals_pop_race['Count_counts']/totals_pop_race['Count_totals']*100, 1).replace(np.nan, 0))]

,EC_Name,Count_totals,EC_ID,Tier,Parent,Variable,Category,Count_counts,Percent,Reginal_Counts,Reginal_Percents,Geometry


In [41]:
# compare with geojson
print(pop_race_v2.shape)
(pop_race_v2[pop_race_v2.columns[:-1]] == pop_race_geojson[pop_race_geojson.columns[:-1]]).sum()

(1056, 11)


EC_ID               1056
EC_Name             1056
Tier                1056
Parent              1056
Variable            1056
Category            1056
Count               1056
Percent             1056
Reginal_Counts      1056
Reginal_Percents    1056
dtype: int64

## 5. Household Population

In [42]:
pop_households_v1 = pd.read_csv(r"J:\DataScience\DSEconProdDessem\EC2\Dem_Data by_EC yr_2022\Outputs\pop_households.csv")
pop_households_v1.head()

,EC_ID,EC_Name,Tier,Parent,Pop_HH,GQ_Mil,GQ_Col,GQ_Oth,Pop_GQ,Pop_Total,%Pop_HH,%GQ_Mil,%GQ_Col,%GQ_Oth,%Pop_GQ
0,1,San Diego Airport,3,0,5524,0,0,241,241,5765,95.8,0.0,0.0,4.2,4.2
1,2,Alpine,4,0,4255,0,0,109,109,4364,97.5,0.0,0.0,2.5,2.5
2,3,Barrio Logan,4,0,4420,0,0,646,646,5066,87.2,0.0,0.0,12.8,12.8
3,4,Carlsbad Palomar Airport,2,0,928,0,0,83,83,1011,91.8,0.0,0.0,8.2,8.2
4,5,Carlsbad State Beach,3,0,831,0,0,0,0,831,100.0,0.0,0.0,0.0,0.0


In [43]:
pop_households_v2 = pd.read_csv(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Dem-Data_by_EC_yr-2022\Outputs\.csv files\pop_households.csv')
pop_households_v2.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Reginal_Counts,Reginal_Percents,Geometry
0,1,San Diego Airport,3,0,Pop_HH,Pop by Type,5524,95.8,3170895.0,96.5,MULTIPOLYGON (((6266765.922053143 1846488.1797...
1,2,Alpine,4,0,Pop_HH,Pop by Type,4255,97.5,3170895.0,96.5,MULTIPOLYGON (((6409937.99988164 1889070.00005...
2,3,Barrio Logan,4,0,Pop_HH,Pop by Type,4420,87.2,3170895.0,96.5,MULTIPOLYGON (((6286603.12483272 1836426.15324...
3,4,Carlsbad Palomar Airport,2,0,Pop_HH,Pop by Type,928,91.8,3170895.0,96.5,MULTIPOLYGON (((6252306.248872802 1998205.2030...
4,5,Carlsbad State Beach,3,0,Pop_HH,Pop by Type,831,100.0,3170895.0,96.5,MULTIPOLYGON (((6231083.038038477 1994552.5840...


In [44]:
pop_households_geojson = gp.read_file(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Dem-Data_by_EC_yr-2022\Outputs\.geojson files\pop_households.geojson')
pop_households_geojson.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Reginal_Counts,Reginal_Percents,geometry
0,1,San Diego Airport,3,0,Pop_HH,Pop by Type,5524,95.8,3170895.0,96.5,"MULTIPOLYGON (((6266765.92205 1846488.17971, 6..."
1,2,Alpine,4,0,Pop_HH,Pop by Type,4255,97.5,3170895.0,96.5,"MULTIPOLYGON (((6409937.99988 1889070.00006, 6..."
2,3,Barrio Logan,4,0,Pop_HH,Pop by Type,4420,87.2,3170895.0,96.5,"MULTIPOLYGON (((6286603.12483 1836426.15325, 6..."
3,4,Carlsbad Palomar Airport,2,0,Pop_HH,Pop by Type,928,91.8,3170895.0,96.5,"MULTIPOLYGON (((6252306.24887 1998205.20305, 6..."
4,5,Carlsbad State Beach,3,0,Pop_HH,Pop by Type,831,100.0,3170895.0,96.5,"MULTIPOLYGON (((6231083.03804 1994552.58404, 6..."


In [45]:
# check all ECs in v2
for ec in pop_households_v1['EC_Name'].unique():
    if ec not in pop_households_v2['EC_Name'].unique():
        print(ec)

# check all variables in v2
for var in pop_households_v1.columns:
    if var not in pop_households_v2['Variable'].unique():
        if '_'.join(var.split('_')[1:]) not in pop_households_v2['Variable'].unique():
            print(var)

Carlsbad Palomar Airport Sub-Center: Airport
Carlsbad Palomar Airport Sub-Center: El Camino Real
Carlsbad Palomar Airport Sub-Center: Faraday
Carlsbad Palomar Airport Sub-Center: Melrose
Carlsbad Palomar Airport Sub-Center: Palomar Oaks
Kearny Mesa East Sub-Center: Aero Dr
Kearny Mesa East Sub-Center: Balboa Ave
Kearny Mesa East Sub-Center: Cabrillo Hts
Kearny Mesa East Sub-Center: I-15 Border
Kearny Mesa East Sub-Center: Montgomery-Gibbs Exec Airport
Kearny Mesa East Sub-Center: North Point
Kearny Mesa East Sub-Center: Spectrum Ctr
Sorrento Valley West Sub-Center: Scripps Research
EC_ID
EC_Name
Tier
Parent
Pop_Total
%Pop_HH
%GQ_Mil
%GQ_Col
%GQ_Oth
%Pop_GQ


In [46]:
# check totals by EC
pop_households_v1[pop_households_v1['EC_Name'].isin(pop_households_v2['EC_Name'].unique())].sort_values('EC_Name')[pop_households_v1.columns[4:9]].sum(axis=1).to_list() == pop_households_v2.groupby(['EC_Name', 'Tier', 'Parent']).sum()['Count'].to_list()

True

In [47]:
# check totals by EC
pop_households_v1[pop_households_v1['EC_Name'].isin(pop_households_v2['EC_Name'].unique())].sort_values('EC_Name')['Pop_Total'].to_list() == pop_households_v2[pop_households_v2['Variable'].isin(pop_households_v1.columns[4:8])].groupby(['EC_Name', 'Tier', 'Parent']).sum()['Count'].to_list()

True

In [48]:
# check totals by variable
pop_households_v2.groupby('Variable')['Count'].sum().sort_index().to_list() == pop_households_v1[pop_households_v1['EC_Name'].isin(pop_households_v2['EC_Name'].unique())][pop_households_v1.columns[4:9]].sum().sort_index().to_list()

True

In [49]:
# check percentages
totals_pop_households = pop_households_v2[pop_households_v2['Variable'].isin(['Pop_HH', 'Pop_GQ'])].groupby('EC_Name')['Count'].sum().reset_index().merge(pop_households_v2, on='EC_Name', how='outer', suffixes=('_totals', '_counts'))
totals_pop_households['Percentage_check'] = round(totals_pop_households['Count_counts']/totals_pop_households['Count_totals']*100, 1).replace(np.nan, 0)
totals_pop_households[~(totals_pop_households['Percent'] == totals_pop_households['Percentage_check'])]

,EC_Name,Count_totals,EC_ID,Tier,Parent,Variable,Category,Count_counts,Percent,Reginal_Counts,Reginal_Percents,Geometry,Percentage_check


In [50]:
# compare with geojson
print(pop_households_v2.shape)
(pop_households_v2[pop_households_v2.columns[:-1]] == pop_households_geojson[pop_households_geojson.columns[:-1]]).sum()

(660, 11)


EC_ID               660
EC_Name             660
Tier                660
Parent              660
Variable            660
Category            660
Count               660
Percent             660
Reginal_Counts      660
Reginal_Percents    660
dtype: int64

# ---------------------------------
# Emp_Counts_yr-2022

In [51]:
emp_v1 = pd.read_csv(r"J:\DataScience\DSEconProdDessem\EC2\Emp_Counts_2022\Outputs\ECs_Emp_Counts_yr_2022.csv")
emp_v1.head()

,EC_ID,emp_gov,emp_mil,emp_ag_min,emp_bus_svcs,emp_fin_res_mgm,emp_educ,emp_hlth,emp_ret,emp_trn_wrh_con,...,%emp_hlth,%emp_ret,%emp_trn_wrh_con,%emp_utl_mnf_whl,%emp_ent,%emp_accm,%emp_food,%emp_oth,%emp_non_ws_wfh,%emp_non_ws_oth
0,0,11978,1921,2702,25509,9863,47366,35118,15227,21453,...,7.6,3.3,4.6,2.1,1.3,1.0,4.0,2.5,19.3,32.6
1,1,1105,0,7,3548,1439,707,682,889,4694,...,2.8,3.7,19.5,3.3,1.4,3.9,12.3,3.7,1.2,19.9
2,2,239,0,0,165,73,36,375,259,287,...,14.4,10.0,11.0,1.8,0.5,0.4,13.7,1.9,7.7,18.8
3,3,87,0,0,418,48,422,558,382,513,...,10.4,7.1,9.6,21.2,0.0,0.1,11.5,2.1,2.7,17.0
4,4,874,0,20,9446,3032,971,1813,2084,2367,...,3.8,4.4,5.0,37.9,1.0,0.7,1.7,0.8,0.1,14.0


In [52]:
emp_v2 = pd.read_csv(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Emp_Counts_2022\Outputs\emp_count_2022.csv')
emp_v2.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Regional_Count,Regional_Percent,Geometry
0,1,San Diego Airport,3,0,emp_gov,Emps by Industry,1105,4.6,121320.0,5.7,MULTIPOLYGON (((6266765.922053143 1846488.1797...
1,2,Alpine,4,0,emp_gov,Emps by Industry,239,9.2,121320.0,5.7,MULTIPOLYGON (((6409937.99988164 1889070.00005...
2,3,Barrio Logan,4,0,emp_gov,Emps by Industry,87,1.6,121320.0,5.7,MULTIPOLYGON (((6286603.12483272 1836426.15324...
3,4,Carlsbad Palomar Airport,2,0,emp_gov,Emps by Industry,874,1.9,121320.0,5.7,MULTIPOLYGON (((6252306.248872802 1998205.2030...
4,5,Carlsbad State Beach,3,0,emp_gov,Emps by Industry,187,1.1,121320.0,5.7,MULTIPOLYGON (((6231083.038038477 1994552.5840...


In [53]:
emp_geojson = gp.read_file(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Emp_Counts_2022\Outputs\emp_count_2022.geojson')
emp_geojson.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Regional_Count,Regional_Percent,geometry
0,1,San Diego Airport,3,0,emp_gov,Emps by Industry,1105,4.6,121320.0,5.7,"MULTIPOLYGON (((6266765.92205 1846488.17971, 6..."
1,2,Alpine,4,0,emp_gov,Emps by Industry,239,9.2,121320.0,5.7,"MULTIPOLYGON (((6409937.99988 1889070.00006, 6..."
2,3,Barrio Logan,4,0,emp_gov,Emps by Industry,87,1.6,121320.0,5.7,"MULTIPOLYGON (((6286603.12483 1836426.15325, 6..."
3,4,Carlsbad Palomar Airport,2,0,emp_gov,Emps by Industry,874,1.9,121320.0,5.7,"MULTIPOLYGON (((6252306.24887 1998205.20305, 6..."
4,5,Carlsbad State Beach,3,0,emp_gov,Emps by Industry,187,1.1,121320.0,5.7,"MULTIPOLYGON (((6231083.03804 1994552.58404, 6..."


In [54]:
# check all ECs in v2
for ec in emp_v1['EC_ID'].unique():
    if ec not in emp_v2['EC_ID'].unique():
        print(ec)

# check all variables in v2
for var in emp_v1.columns:
    if var not in emp_v2['Variable'].unique():
        print(var)

0
EC_ID
emp_tot
%emp_gov
%emp_mil
%emp_ag_min
%emp_bus_svcs
%emp_fin_res_mgm
%emp_educ
%emp_hlth
%emp_ret
%emp_trn_wrh_con
%emp_utl_mnf_whl
%emp_ent
%emp_accm
%emp_food
%emp_oth
%emp_non_ws_wfh
%emp_non_ws_oth


In [55]:
# check totals by EC
emp_v1[emp_v1['EC_ID'].isin(emp_v2['EC_ID'].unique())].sort_values('EC_ID')['emp_tot'].to_list() == emp_v2.groupby(['EC_ID', 'Tier', 'Parent']).sum()['Count'].to_list()

True

In [56]:
# check totals by variable
emp_v2.groupby('Variable')['Count'].sum().sort_index() == emp_v1[emp_v1['EC_ID'].isin(emp_v2['EC_ID'].unique())][emp_v2['Variable'].unique()].sum().sort_index()

Variable
emp_accm           True
emp_ag_min         True
emp_bus_svcs       True
emp_educ           True
emp_ent            True
emp_fin_res_mgm    True
emp_food           True
emp_gov            True
emp_hlth           True
emp_mil            True
emp_non_ws_oth     True
emp_non_ws_wfh     True
emp_oth            True
emp_ret            True
emp_trn_wrh_con    True
emp_utl_mnf_whl    True
dtype: bool

In [57]:
# compare with geojson
print(emp_v2.shape)
(emp_v2[emp_v2.columns[:-1]] == emp_geojson[emp_geojson.columns[:-1]]).sum()

(1632, 11)


EC_ID               1632
EC_Name             1632
Tier                1632
Parent              1632
Variable            1632
Category            1632
Count               1632
Percent             1632
Regional_Count      1632
Regional_Percent    1632
dtype: int64

In [58]:
emp_v2.groupby('Variable')['Count'].sum()

Variable
emp_accm            14048
emp_ag_min            603
emp_bus_svcs       241761
emp_educ            83552
emp_ent             25248
emp_fin_res_mgm     95839
emp_food           121644
emp_gov            109342
emp_hlth           164778
emp_mil            108079
emp_non_ws_oth     257335
emp_non_ws_wfh      38892
emp_oth             39154
emp_ret            128958
emp_trn_wrh_con     88115
emp_utl_mnf_whl    165364
Name: Count, dtype: int64

In [59]:
emp_v2[['Variable', 'Regional_Count']].drop_duplicates()

,Variable,Regional_Count
0,emp_gov,121320.0
102,emp_mil,110000.0
204,emp_ag_min,3305.0
306,emp_bus_svcs,267270.0
408,emp_fin_res_mgm,105702.0
510,emp_educ,130918.0
612,emp_hlth,199896.0
714,emp_ret,144185.0
816,emp_trn_wrh_con,109568.0
918,emp_utl_mnf_whl,175188.0


# ---------------------------------
# LEHD_OD_8_YR_2019

## Change observations:
1. Query has changed:

2020:

```
'''
select *
from [socioec_data].[lehd_lodes].[od_8_0]
where yr = 2020 and (type = 'JT00' or type = 'JT01')
'''
```

2019:
```
'''
select *
from [socioec_data].[lehd_lodes].[od_8_0]
where yr = 2019 and type = 'JT00' and h_geocode like '06073%'
'''
```

In [60]:
od_v1 = pd.read_csv(r'J:\DataScience\DSEconProdDessem\EC2\LEHD OD_8_0 yr_2020\Outputs\jobs_all.csv')
od_v1.head()

,EC_ID,EC_Name,Tier,Home_Block,All_Jobs_Total,All_Jobs_age_29_or_younger,All_Jobs_age_30to54,All_Jobs_age_55_or_older,All_Jobs_jobs_earning_1250_or_less,All_Jobs_jobs_earning_1251_to_3333,All_Jobs_jobs_earning_3333_plus,All_Jobs_Goods_Prod_ind_sect,All_Jobs_Trade_Transport_Utilities_ind_sect,All_Jobs_All_Other_Serv_ind_sect
0,1,San Diego Airport,3,10310105005018,1,0,1,0,1,0,0,0,0,1
1,1,San Diego Airport,3,40030006002069,1,0,1,0,0,0,1,0,1,0
2,1,San Diego Airport,3,40030013023025,1,1,0,0,1,0,0,0,1,0
3,1,San Diego Airport,3,40050005003017,1,0,0,1,1,0,0,0,1,0
4,1,San Diego Airport,3,40050011032007,1,0,0,1,0,0,1,0,1,0


In [61]:
od_v2 = pd.read_csv(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\LEHD_OD_8_YR_2019\Outputs\od_all-jobs_2019.csv')
od_v2.head()

,EC_ID,EC_Name,Tier,Parent,Origin,Variable,Category,Count,Percent,Regional_Count,Regional_Percent,Geometry
0,1,San Diego Airport,3,0,60730001001000,Job,All jobs,1,0.007,0,0,MULTIPOLYGON (((6273729.00359939 1857092.01906...
1,1,San Diego Airport,3,0,60730001001000,Job,All jobs,1,0.007,0,0,MULTIPOLYGON (((6273729.00359939 1857092.01906...
2,4,Carlsbad Palomar Airport,2,0,60730001001000,Job,All jobs,1,0.004,0,0,MULTIPOLYGON (((6273729.00359939 1857092.01906...
3,5,Carlsbad State Beach,3,0,60730001001000,Job,All jobs,1,0.012,0,0,MULTIPOLYGON (((6273729.00359939 1857092.01906...
4,7,Carmel Valley,3,0,60730001001000,Job,All jobs,2,0.015,0,0,MULTIPOLYGON (((6273729.00359939 1857092.01906...


In [62]:
od_geojson = gp.read_file(r'J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\LEHD_OD_8_YR_2019\Outputs\od_all_jobs_2019.geojson')
od_geojson.head()

,EC_ID,EC_Name,Tier,Parent,Origin,Variable,Category,Count,Percent,Regional_Count,Regional_Percent,geometry
0,1,San Diego Airport,3,0,60730001001000,Job,All jobs,1,0.007,0,0,"MULTIPOLYGON (((6273729.00360 1857092.01907, 6..."
1,1,San Diego Airport,3,0,60730001001000,Job,All jobs,1,0.007,0,0,"MULTIPOLYGON (((6273729.00360 1857092.01907, 6..."
2,4,Carlsbad Palomar Airport,2,0,60730001001000,Job,All jobs,1,0.004,0,0,"MULTIPOLYGON (((6273729.00360 1857092.01907, 6..."
3,5,Carlsbad State Beach,3,0,60730001001000,Job,All jobs,1,0.012,0,0,"MULTIPOLYGON (((6273729.00360 1857092.01907, 6..."
4,7,Carmel Valley,3,0,60730001001000,Job,All jobs,1,0.008,0,0,"MULTIPOLYGON (((6273729.00360 1857092.01907, 6..."


In [63]:
# check all ECs in v2
for ec in od_v1['EC_Name'].unique():
    if ec not in od_v2['EC_Name'].unique():
        print(ec)

# check all variables in v2
for var in od_v1.columns:
    if var not in od_v2['Variable'].unique():
        if 'All_' + '_'.join(var.split('_')[2:]) not in od_v2['Variable'].unique():
            print(var)

EC_ID
EC_Name
Tier
Home_Block
All_Jobs_Total
All_Jobs_age_29_or_younger
All_Jobs_age_30to54
All_Jobs_age_55_or_older
All_Jobs_jobs_earning_1250_or_less
All_Jobs_jobs_earning_1251_to_3333
All_Jobs_jobs_earning_3333_plus
All_Jobs_Goods_Prod_ind_sect
All_Jobs_Trade_Transport_Utilities_ind_sect
All_Jobs_All_Other_Serv_ind_sect


In [64]:
od_v2['Variable'].unique()

array(['Job'], dtype=object)

In [65]:
# compare with geojson
print(od_v2.shape)
(od_v2[od_v2.columns[:-1]] == od_geojson[od_geojson.columns[:-1]]).sum()

(790348, 12)


EC_ID               790348
EC_Name             790348
Tier                790348
Parent              790348
Origin              790348
Variable            790348
Category            790348
Count               684577
Percent             684577
Regional_Count      790348
Regional_Percent    790348
dtype: int64

# ---------------------------------
# LEHD_wac_8_0_yr_2019

In [66]:
wac_v1 = pd.read_csv(r"J:\DataScience\DSEconProdDessem\EC2\LEHD_wac_8_0_yr_2019\Outputs\LEHD_wac_8_0_yr_2019.csv")
wac_v1.head()

,EC_ID,EC_Name,Tier,All_Jobs_Total,All_Jobs_male,All_Jobs_female,All_Jobs_age_lt30,All_Jobs_age_30to54,All_Jobs_age_55plus,All_Jobs_educ30_lt_hs,...,Priv_Jobs_%_jobs_firms_age_0_to_1,Priv_Jobs_%_jobs_firms_age_2_to_3,Priv_Jobs_%_jobs_firms_age_4_to_5,Priv_Jobs_%_jobs_firms_age_6_to_10,Priv_Jobs_%_jobs_firms_age_11_plus,Priv_Jobs_%_jobs_firms_size_0_to_19,Priv_Jobs_%_jobs_firms_size_20_to_49,Priv_Jobs_%_jobs_firms_size_50_to_249,Priv_Jobs_%_jobs_firms_size_250_to_499,Priv_Jobs_%_jobs_firms_size_500_plus
0,1,San Diego Airport,3,18582,10517,8065,4459,9964,4159,2575,...,2.40,5.37,4.17,9.60,78.46,15.12,13.01,18.86,4.48,48.53
1,2,Alpine,4,1227,540,687,324,628,275,157,...,1.75,8.03,5.41,18.08,66.72,33.89,24.54,29.78,0.26,11.53
2,3,Barrio Logan,4,4237,2557,1680,936,2353,948,933,...,3.32,5.45,2.79,5.35,83.09,20.45,17.31,11.00,2.94,48.30
3,4,Carlsbad Palomar Airport,2,32826,18358,14468,6451,19376,6999,4032,...,2.30,4.71,2.61,9.19,81.19,14.11,11.15,19.43,7.45,47.86
4,5,Carlsbad State Beach,3,11545,6089,5456,3839,5482,2224,1262,...,0.81,1.51,2.92,8.87,85.90,8.75,5.90,15.08,10.56,59.70


## 1. All jobs

In [67]:
wac_all_v2 = pd.read_csv(r"J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\LEHD_wac_8_0_yr_2019\Outputs\.csv files\wac_all-jobs_2019.csv")
wac_all_v2.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Regional_Count,Regional_Percent,Geometry
0,1,San Diego Airport,3,0,All_male,Jobs by Gender,10517,56.60,742831.0,51.2,MULTIPOLYGON (((6266765.922053143 1846488.1797...
1,2,Alpine,4,0,All_male,Jobs by Gender,540,44.01,742831.0,51.2,MULTIPOLYGON (((6409937.99988164 1889070.00005...
2,3,Barrio Logan,4,0,All_male,Jobs by Gender,2557,60.35,742831.0,51.2,MULTIPOLYGON (((6286603.12483272 1836426.15324...
3,4,Carlsbad Palomar Airport,2,0,All_male,Jobs by Gender,18358,55.93,742831.0,51.2,MULTIPOLYGON (((6252306.248872802 1998205.2030...
4,5,Carlsbad State Beach,3,0,All_male,Jobs by Gender,6089,52.74,742831.0,51.2,MULTIPOLYGON (((6231083.038038477 1994552.5840...


In [68]:
wac_all_geojson = gp.read_file(r"J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\LEHD_wac_8_0_yr_2019\Outputs\.geojson files\wac_all_jobs_2019.geojson")
wac_all_geojson.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Regional_Count,Regional_Percent,geometry
0,1,San Diego Airport,3,0,All_male,Jobs by Gender,10517,56.60,742831.0,51.2,"MULTIPOLYGON (((6266765.92205 1846488.17971, 6..."
1,2,Alpine,4,0,All_male,Jobs by Gender,540,44.01,742831.0,51.2,"MULTIPOLYGON (((6409937.99988 1889070.00006, 6..."
2,3,Barrio Logan,4,0,All_male,Jobs by Gender,2557,60.35,742831.0,51.2,"MULTIPOLYGON (((6286603.12483 1836426.15325, 6..."
3,4,Carlsbad Palomar Airport,2,0,All_male,Jobs by Gender,18358,55.93,742831.0,51.2,"MULTIPOLYGON (((6252306.24887 1998205.20305, 6..."
4,5,Carlsbad State Beach,3,0,All_male,Jobs by Gender,6089,52.74,742831.0,51.2,"MULTIPOLYGON (((6231083.03804 1994552.58404, 6..."


In [69]:
wac_v1.rename(columns={'All_Jobs_Total':'All_Total',
 'All_Jobs_male':'All_male',
 'All_Jobs_female':'All_female',
 'All_Jobs_age_lt30':'All_age_lt30',
 'All_Jobs_age_30to54':'All_age_30to54',
 'All_Jobs_age_55plus':'All_age_55plus',
 'All_Jobs_educ30_lt_hs':'All_ed30_lt_hs',
 'All_Jobs_educ30_hs':'All_ed30_hs',
 'All_Jobs_educ30_some_college':'All_ed30_some_coll',
 'All_Jobs_educ30_bachelor_plus':'All_ed30_bach_plus',
 'All_Jobs_jobs_earning_1250_or_less':'All_earn_upto_1250',
 'All_Jobs_jobs_earning_1251_to_3333':'All_earn_1251-3333',
 'All_Jobs_jobs_earning_3333_plus':'All_earn_3333_plus',
 'All_Jobs_%_male':'All_%_male',
 'All_Jobs_%_female':'All_%_female',
 'All_Jobs_%_age_lt30':'All_%_age_lt30',
 'All_Jobs_%_age_30to54':'All_%_age_30to54',
 'All_Jobs_%_age_55plus': 'All_%_age_55plus',
 'All_Jobs_%_educ30_lt_hs':'All_%_ed30_lt_hs',
 'All_Jobs_%_Jobs_educ30_hs':'All_%_ed30_hs',
 'All_Jobs_%_educ30_some_college':'All_%_ed30_some_coll',
 'All_Jobs_%_educ30_bachelor_plus':'All_%_ed30_bach_plus',
 'All_Jobs_%_jobs_earning_1250_or_less':'All_%_earn_upto_1250',
 'All_Jobs_%_jobs_earning_1251_to_3333':'All_%_earn_1251-3333',
 'All_Jobs_%_jobs_earning_3333_plus':'All_%_earn_3333_plus',
 'Priv_Jobs_Total':'Priv_Total',
 'Priv_Jobs_male':'Priv_male',
 'Priv_Jobs_female':'Priv_female',
 'Priv_Jobs_age_lt30':'Priv_age_lt30',
 'Priv_Jobs_age_30to54':'Priv_age_30to54',
 'Priv_Jobs_age_55plus':'Priv_age_55plus',
 'Priv_Jobs_educ30_lt_hs':'Priv_ed30_lt_hs',
 'Priv_Jobs_educ30_hs':'Priv_ed30_hs',
 'Priv_Jobs_educ30_some_college':'Priv_ed30_some_coll',
 'Priv_Jobs_educ30_bachelor_plus':'Priv_ed30_bach_plus',
 'Priv_Jobs_jobs_earning_1250_or_less':'Priv_earn_upto_1250',
 'Priv_Jobs_jobs_earning_1251_to_3333':'Priv_earn_1251-3333',
 'Priv_Jobs_jobs_earning_3333_plus':'Priv_earn_3333_plus',
 'Priv_Jobs_%_male':'Priv_%_male',
 'Priv_Jobs_%_female':'Priv_%_female',
 'Priv_Jobs_%_age_lt30':'Priv_%_age_lt30',
 'Priv_Jobs_%_age_30to54':'Priv_%_age_30to54',
 'Priv_Jobs_%_age_55plus':'Priv_%_age_55plus',
 'Priv_Jobs_%_educ30_lt_hs':'Priv_%_ed30_lt_hs',
 'Priv_Jobs_%_Jobs_educ30_hs':'Priv_%_ed30_hs',
 'Priv_Jobs_%_educ30_some_college':'Priv_%_ed30_some_coll',
 'Priv_Jobs_%_educ30_bachelor_plus':'Priv_%_ed30_bach_plus',
 'Priv_Jobs_%_jobs_earning_1250_or_less':'Priv_%_earn_upto_1250',
 'Priv_Jobs_%_jobs_earning_1251_to_3333':'Priv_%_earn_1251-3333',
 'Priv_Jobs_%_jobs_earning_3333_plus':'Priv_%_earn_3333_plus'
 },inplace=True)

In [70]:
for ec in wac_v1['EC_Name'].unique():
    if ec not in wac_all_v2['EC_Name'].unique():
        print(ec)

Kearny Mesa
Sorrento Valley


In [71]:
# check totals by EC
wac_v1[wac_v1['EC_Name'].isin(wac_all_v2['EC_Name'].unique())].sort_values('EC_Name')[wac_v1.columns[4:16]].sum(axis=1).to_list() == wac_all_v2.groupby(['EC_Name', 'Tier', 'Parent']).sum()['Count'].to_list()

True

In [72]:
# check totals by variable
wac_all_v2.groupby('Variable')['Count'].sum().sort_index() == wac_v1[wac_v1['EC_Name'].isin(wac_all_v2['EC_Name'].unique())][wac_all_v2['Variable'].unique()].sum().sort_index()

Variable
All_age_30to54        True
All_age_55plus        True
All_age_lt30          True
All_earn_1251-3333    True
All_earn_3333_plus    True
All_earn_upto_1250    True
All_ed30_bach_plus    True
All_ed30_hs           True
All_ed30_lt_hs        True
All_ed30_some_coll    True
All_female            True
All_male              True
dtype: bool

In [73]:
# compare with geojson
print(wac_all_v2.shape)
(wac_all_v2[emp_v2.columns[:-1]] == wac_all_geojson[wac_all_geojson.columns[:-1]]).sum()

(1728, 11)


EC_ID               1728
EC_Name             1728
Tier                1728
Parent              1728
Variable            1728
Category            1728
Count               1728
Percent             1728
Regional_Count      1728
Regional_Percent    1728
dtype: int64

## 2. Private jobs

In [74]:
wac_private_v2 = pd.read_csv(r"J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\LEHD_wac_8_0_yr_2019\Outputs\.csv files\wac_private-jobs_2019.csv")
wac_private_v2.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Regional_Count,Regional_Percent,Geometry
0,1,San Diego Airport,3,0,Priv_male,Jobs by Gender,9731.0,57.04,664493.0,53.1,MULTIPOLYGON (((6266765.922053143 1846488.1797...
1,2,Alpine,4,0,Priv_male,Jobs by Gender,515.0,44.98,664493.0,53.1,MULTIPOLYGON (((6409937.99988164 1889070.00005...
2,3,Barrio Logan,4,0,Priv_male,Jobs by Gender,2466.0,61.94,664493.0,53.1,MULTIPOLYGON (((6286603.12483272 1836426.15324...
3,4,Carlsbad Palomar Airport,2,0,Priv_male,Jobs by Gender,18338.0,55.93,664493.0,53.1,MULTIPOLYGON (((6252306.248872802 1998205.2030...
4,5,Carlsbad State Beach,3,0,Priv_male,Jobs by Gender,6066.0,52.68,664493.0,53.1,MULTIPOLYGON (((6231083.038038477 1994552.5840...


In [75]:
wac_private_geojson = gp.read_file(r"J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\LEHD_wac_8_0_yr_2019\Outputs\.geojson files\wac_private_jobs_2019.geojson")
wac_private_geojson.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Regional_Count,Regional_Percent,geometry
0,1,San Diego Airport,3,0,Priv_male,Jobs by Gender,9731.0,57.04,664493.0,53.1,"MULTIPOLYGON (((6266765.92205 1846488.17971, 6..."
1,2,Alpine,4,0,Priv_male,Jobs by Gender,515.0,44.98,664493.0,53.1,"MULTIPOLYGON (((6409937.99988 1889070.00006, 6..."
2,3,Barrio Logan,4,0,Priv_male,Jobs by Gender,2466.0,61.94,664493.0,53.1,"MULTIPOLYGON (((6286603.12483 1836426.15325, 6..."
3,4,Carlsbad Palomar Airport,2,0,Priv_male,Jobs by Gender,18338.0,55.93,664493.0,53.1,"MULTIPOLYGON (((6252306.24887 1998205.20305, 6..."
4,5,Carlsbad State Beach,3,0,Priv_male,Jobs by Gender,6066.0,52.68,664493.0,53.1,"MULTIPOLYGON (((6231083.03804 1994552.58404, 6..."


In [76]:
for ec in wac_v1['EC_Name'].unique():
    if ec not in wac_private_v2['EC_Name'].unique():
        print(ec)

Kearny Mesa
Sorrento Valley


In [77]:
# check totals by EC
wac_private_v2.groupby(['EC_Name', 'Tier', 'Parent']).sum()['Count'].to_list() == wac_v1[wac_v1['EC_Name'].isin(wac_private_v2['EC_Name'].unique())].sort_values('EC_Name')[wac_v1.columns[29:41]].sum(axis=1).to_list()

False

In [78]:
# check totals by variable
wac_private_v2.groupby('Variable')['Count'].sum().sort_index() == wac_v1[wac_v1['EC_Name'].isin(wac_private_v2['EC_Name'].unique())][wac_private_v2['Variable'].unique()].sum().sort_index()

Variable
Priv_age_30to54        True
Priv_age_55plus        True
Priv_age_lt30          True
Priv_earn_1251-3333    True
Priv_earn_3333_plus    True
Priv_earn_upto_1250    True
Priv_ed30_bach_plus    True
Priv_ed30_hs           True
Priv_ed30_lt_hs        True
Priv_ed30_some_coll    True
Priv_female            True
Priv_male              True
dtype: bool

In [79]:
# compare with geojson
print(wac_private_v2.shape)
(wac_private_v2[emp_v2.columns[:-1]] == wac_private_geojson[wac_private_geojson.columns[:-1]]).sum()

(1728, 11)


EC_ID               1728
EC_Name             1728
Tier                1728
Parent              1728
Variable            1728
Category            1728
Count               1716
Percent             1716
Regional_Count      1728
Regional_Percent    1728
dtype: int64

# Replica 2022

In [80]:
replica_dict = {
    'Mode_auto_passenger':'Mode_auto_psgr',
    'Mode_on_demand_auto':'Mode_on_dem_auto',
    'Mode_other_travel_mode':'Mode_other',
    'Trip_Purp_commercial':'Trip_Purp_coml',
    'Trip_Purp_maintenance':'Trip_Purp_maint',
    'Trip_Purp_other_activity_type':'Trip_Purp_other',
    'Trip_Purp_region_departure':'Trip_Purp_regn_dept',
    'Prv_Trip_Purp_commercial':'Prv_Trip_Purp_coml',
    'Prv_Trip_Purp_maintenance':'Prv_Trip_Purp_maint',
    'Prv_Trip_Purp_other_activity_type':'Prv_Trip_Purp_other',
    'Prv_Trip_Purp_recreation':'Prv_Trip_Purp_rec',
    'Veh_Type_heavy_commercial':'Veh_Type_heavy_coml',
    'Veh_Type_medium_commercial':'Veh_Type_med_coml',
    'Veh_Type_unknown_vehicle_type':'Veh_Type_unknown',
    'Veh_Type_passenger_car':'Veh_Type_psgr_car',
    'Trip_Dist_less_than_1_mile':'Dist_0-1_mile',
 'Trip_Dist_1-1.99_miles':'Dist_1-1.99_miles',
 'Trip_Dist_2-4.99_miles':'Dist_2-4.99_miles',
 'Trip_Dist_5-9.99_miles':'Dist_5-9.99_miles',
 'Trip_Dist_10-24.99_miles':'Dist_10-24.99_miles',
 'Trip_Dist_25+_miles':'Dist_25+_miles',
    'Trip_Dur_0-4.99_min':'Dur_0-4.99_min',
 'Trip_Dur_5-14.99_min':'Dur_5-14.99_min',
 'Trip_Dur_15-29.99_min':'Dur_15-29.99_min',
 'Trip_Dur_30-59.99_min':'Dur_30-59.99_min',
 'Trip_Dur_60+_min':'Dur_60+_min',
    'Orig_Bld_Use_agriculture':'Orig_Bld_Use_agr',
 'Orig_Bld_Use_civic_institutional':'Orig_Bld_Use_civic',
 'Orig_Bld_Use_education':'Orig_Bld_Use_educ',
 'Orig_Bld_Use_healthcare':'Orig_Bld_Use_health',
 'Orig_Bld_Use_industrial':'Orig_Bld_Use_indus',
 'Orig_Bld_Use_multi_family':'Orig_Bld_Use_mul_fam',
 'Orig_Bld_Use_non_retail_attraction':'Orig_Bld_Use_non_ret',
 'Orig_Bld_Use_office':'Orig_Bld_Use_office',
 'Orig_Bld_Use_open_space':'Orig_Bld_Use_open',
 'Orig_Bld_Use_other':'Orig_Bld_Use_other',
 'Orig_Bld_Use_retail':'Orig_Bld_Use_retail',
 'Orig_Bld_Use_single_family':'Orig_Bld_Use_sgl_fam',
 'Orig_Bld_Use_transportation_utilities':'Orig_Bld_Use_trans',
 'Orig_Bld_Use_unknown':'Orig_Bld_Use_unknown',
 'Orig_Land_Use_agriculture':'Orig_Land_Use_agr',
 'Orig_Land_Use_civic_institutional':'Orig_Land_Use_civic',
 'Orig_Land_Use_education':'Orig_Land_Use_educ',
 'Orig_Land_Use_healthcare':'Orig_Land_Use_health',
 'Orig_Land_Use_industrial':'Orig_Land_Use_indus',
 'Orig_Land_Use_mixed_use':'Orig_Land_Use_mix_use',
 'Orig_Land_Use_multi_family':'Orig_Land_Use_mul_fam',
 'Orig_Land_Use_non_retail_attraction':'Orig_Land_Use_non_ret',
 'Orig_Land_Use_office':'Orig_Land_Use_office',
 'Orig_Land_Use_open_space':'Orig_Land_Use_open',
 'Orig_Land_Use_other':'Orig_Land_Use_other',
 'Orig_Land_Use_retail':'Orig_Land_Use_retail',
 'Orig_Land_Use_single_family':'Orig_Land_Use_sgl_fam',
 'Orig_Land_Use_transportation_utilities': 'Orig_Land_Use_trans',
 'Orig_Land_Use_unknown':'Orig_Land_Use_unknown',
    'Dest_Bld_Use_agriculture':'Dest_Bld_Use_agr',
 'Dest_Bld_Use_civic_institutional':'Dest_Bld_Use_civic',
 'Dest_Bld_Use_education':'Dest_Bld_Use_educ',
 'Dest_Bld_Use_healthcare':'Dest_Bld_Use_health',
 'Dest_Bld_Use_industrial':'Dest_Bld_Use_indus',
 'Dest_Bld_Use_multi_family':'Dest_Bld_Use_mul_fam',
 'Dest_Bld_Use_non_retail_attraction':'Dest_Bld_Use_non_ret',
 'Dest_Bld_Use_office':'Dest_Bld_Use_office',
 'Dest_Bld_Use_other':'Dest_Bld_Use_other',
 'Dest_Bld_Use_retail':'Dest_Bld_Use_retail',
 'Dest_Bld_Use_single_family':'Dest_Bld_Use_sgl_fam',
 'Dest_Bld_Use_transportation_utilities':'Dest_Bld_Use_trans',
 'Dest_Bld_Use_unknown':'Dest_Bld_Use_unknown',
 'Dest_Bld_Use_open_space':'Dest_Bld_Use_open',
 'Dest_Land_agriculture':'Dest_Land_Use_agr',
 'Dest_Land_civic_institutional':'Dest_Land_Use_civic',
 'Dest_Land_education':'Dest_Land_Use_educ',
 'Dest_Land_healthcare':'Dest_Land_Use_health',
 'Dest_Land_industrial':'Dest_Land_Use_indus',
 'Dest_Land_mixed_use':'Dest_Land_Use_mix_use',
 'Dest_Land_multi_family':'Dest_Land_Use_mul_fam',
 'Dest_Land_non_retail_attraction':'Dest_Land_Use_non_ret',
 'Dest_Land_office':'Dest_Land_Use_office',
 'Dest_Land_other':'Dest_Land_Use_other',
 'Dest_Land_retail':'Dest_Land_Use_retail',
 'Dest_Land_single_family':'Dest_Land_Use_sgl_fam',
 'Dest_Land_transportation_utilities':'Dest_Land_Use_trans',
 'Dest_Land_unknown':'Dest_Land_Use_unknown',
 'Dest_Land_open_space':'Dest_Land_Use_open'
}

In [81]:
# Read the needed tables
trip_counts_wf = pd.read_csv(r"J:\DataScience\DSEconProdDessem\EC2\Replica\Trips to EC2 Fall 2022 Thrsdays\Tables in Wide Format\Outputs_wide-format\trip_counts.csv")
trip_counts_wf = trip_counts_wf.rename(columns=replica_dict)
trip_counts_wf.head()

,EC_ID,EC_Name,EC_Tier,Trips_Total,Mode_auto_psgr,Mode_biking,Mode_commercial,Mode_on_dem_auto,Mode_other,Mode_private_auto,...,Dest_Land_Use_mix_use,Dest_Land_Use_mul_fam,Dest_Land_Use_non_ret,Dest_Land_Use_office,Dest_Land_Use_other,Dest_Land_Use_retail,Dest_Land_Use_sgl_fam,Dest_Land_Use_trans,Dest_Land_Use_unknown,Dest_Land_Use_open
0,1,San Diego Airport,3,135166,21223.0,1665.0,2049.0,3673.0,14215.0,73664.0,...,16982.0,3054.0,8478.0,3578.0,NaN,42129.0,3321.0,44790.0,2049.0,1459.0
1,2,Alpine,4,25533,6891.0,80.0,884.0,54.0,264.0,15056.0,...,3295.0,1061.0,469.0,3398.0,2.0,10025.0,4760.0,159.0,884.0,NaN
2,3,Barrio Logan,4,42638,6439.0,263.0,1277.0,223.0,257.0,25489.0,...,12694.0,1662.0,330.0,297.0,NaN,13847.0,4875.0,555.0,1277.0,670.0
3,4,Carlsbad Palomar Airport,2,74489,19119.0,481.0,3677.0,130.0,368.0,44257.0,...,32782.0,380.0,1016.0,6008.0,NaN,11401.0,602.0,504.0,3677.0,345.0
4,5,Carlsbad State Beach,3,48396,11896.0,312.0,1121.0,160.0,567.0,31129.0,...,5654.0,816.0,2568.0,11166.0,NaN,25244.0,407.0,520.0,1121.0,148.0


In [82]:
# Read the needed tables
trip_percents_wf = pd.read_csv(r"J:\DataScience\DSEconProdDessem\EC2\Replica\Trips to EC2 Fall 2022 Thrsdays\Tables in Wide Format\Outputs_wide-format\trip_percents.csv")
trip_percents_wf.head()

,EC_ID,EC_Name,EC_Tier,Trips_Total,Mode_auto_passenger,Mode_biking,Mode_commercial,Mode_on_demand_auto,Mode_other_travel_mode,Mode_private_auto,...,Dest_Land_mixed_use,Dest_Land_multi_family,Dest_Land_non_retail_attraction,Dest_Land_office,Dest_Land_other,Dest_Land_retail,Dest_Land_single_family,Dest_Land_transportation_utilities,Dest_Land_unknown,Dest_Land_open_space
0,1,San Diego Airport,3,135166,15.7,1.2,1.5,2.7,10.5,54.5,...,12.6,2.3,6.3,2.6,NaN,31.2,2.5,33.1,1.5,1.1
1,2,Alpine,4,25533,27.0,0.3,3.5,0.2,1.0,59.0,...,12.9,4.2,1.8,13.3,0.0,39.3,18.6,0.6,3.5,NaN
2,3,Barrio Logan,4,42638,15.1,0.6,3.0,0.5,0.6,59.8,...,29.8,3.9,0.8,0.7,NaN,32.5,11.4,1.3,3.0,1.6
3,4,Carlsbad Palomar Airport,2,74489,25.7,0.6,4.9,0.2,0.5,59.4,...,44.0,0.5,1.4,8.1,NaN,15.3,0.8,0.7,4.9,0.5
4,5,Carlsbad State Beach,3,48396,24.6,0.6,2.3,0.3,1.2,64.3,...,11.7,1.7,5.3,23.1,NaN,52.2,0.8,1.1,2.3,0.3


In [83]:
replica_2022 = pd.read_csv(r"J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Replica_2022\Outputs\trips_replica_2022.csv")
replica_2022.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Regional_Count,Regional_Percent,Geometry
0,1,San Diego Airport,3,0,Mode_auto_psgr,Trips by Mode,21223.0,15.7,0,0,MULTIPOLYGON (((6266765.922053143 1846488.1797...
1,1,San Diego Airport,3,0,Mode_biking,Trips by Mode,1665.0,1.2,0,0,MULTIPOLYGON (((6266765.922053143 1846488.1797...
2,1,San Diego Airport,3,0,Mode_commercial,Trips by Mode,2049.0,1.5,0,0,MULTIPOLYGON (((6266765.922053143 1846488.1797...
3,1,San Diego Airport,3,0,Mode_on_dem_auto,Trips by Mode,3673.0,2.7,0,0,MULTIPOLYGON (((6266765.922053143 1846488.1797...
4,1,San Diego Airport,3,0,Mode_other,Trips by Mode,14215.0,10.5,0,0,MULTIPOLYGON (((6266765.922053143 1846488.1797...


In [84]:
replica_geojson = gp.read_file(r"J:\DataScience\DSEconProdDessem\EC2\Tables for Sadra\Replica_2022\Outputs\trips_replica_2022.geojson")
replica_geojson.head()

,EC_ID,EC_Name,Tier,Parent,Variable,Category,Count,Percent,Regional_Count,Regional_Percent,geometry
0,1,San Diego Airport,3,0,Mode_auto_psgr,Trips by Mode,21223.0,15.7,0,0,"MULTIPOLYGON (((6266765.92205 1846488.17971, 6..."
1,1,San Diego Airport,3,0,Mode_biking,Trips by Mode,1665.0,1.2,0,0,"MULTIPOLYGON (((6266765.92205 1846488.17971, 6..."
2,1,San Diego Airport,3,0,Mode_commercial,Trips by Mode,2049.0,1.5,0,0,"MULTIPOLYGON (((6266765.92205 1846488.17971, 6..."
3,1,San Diego Airport,3,0,Mode_on_dem_auto,Trips by Mode,3673.0,2.7,0,0,"MULTIPOLYGON (((6266765.92205 1846488.17971, 6..."
4,1,San Diego Airport,3,0,Mode_other,Trips by Mode,14215.0,10.5,0,0,"MULTIPOLYGON (((6266765.92205 1846488.17971, 6..."


## 1. Trip Counts

In [85]:
for ec in trip_counts_wf['EC_Name'].unique():
    if ec not in replica_2022['EC_Name'].unique():
        print(ec)

In [86]:
# check totals by EC
replica_2022.groupby(['EC_Name', 'Tier', 'Parent'])['Count'].sum().to_list() == trip_counts_wf.rename(columns=replica_dict).sort_values('EC_Name')[replica_2022['Variable'].unique()].sum(axis=1).to_list()

True

In [87]:
# check totals by variable
(replica_2022.groupby('Variable')['Count'].sum().sort_index() == trip_counts_wf[replica_2022['Variable'].unique()].sum().sort_index()).to_list()

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [88]:
# compare with geojson
print(replica_2022.shape)
(replica_2022[replica_2022.columns[:-1]].replace(np.nan, 'nan') == replica_geojson[replica_geojson.columns[:-1]].replace(np.nan, 'nan')).sum()

(10914, 11)


EC_ID               10914
EC_Name             10914
Tier                10914
Parent              10914
Variable            10914
Category            10914
Count               10914
Percent             10914
Regional_Count      10914
Regional_Percent    10914
dtype: int64